In [1]:
import spiketoolkit as st
import spikesorters as ss
import spikeextractors as se
import numpy as np
import spikemetrics.metrics as metrics
from spikemetrics.utils import Epoch

15:45:37 [I] klustakwik KlustaKwik2 version 0.2.6


In [2]:
# workshop_path = "/Users/colehurwitz/spiketutorials/Spike_sorting_workshop_2019/"
workshop_path = "/afs/inf.ed.ac.uk/user/s17/s1727843/spiketutorials/Spike_sorting_workshop_2019/"
recording = se.MEArecRecordingExtractor(recording_path= workshop_path + "recordings_36cells_four-tetrodes_30.0_10.0uV_20-06-2019_14_48.h5")


Could not load plane information. Assuming probe is in yz plane


In [3]:
recording = se.load_probe_file(recording, workshop_path + 'tetrode_16.prb')

In [4]:
sorting = ss.run_klusta(recording)

Klusta units are not unique! Using unique unit ids


In [5]:
metric_info = st.validation.validation_tools.get_quality_metric_data(recording, sorting, nPC=3, ms_before=1., ms_after=2., dtype=None, max_num_waveforms=np.inf, 
                                                        max_num_pca_waveforms=np.inf, save_features_props=False, verbose=False, seed=0)

spike_times = metric_info[0]
spike_clusters = metric_info[1]
amplitudes = metric_info[2]
pc_features = metric_info[3]
pc_feature_ind = metric_info[4]

In [6]:
metric_calculator = st.validation.qualitymetrics.MetricCalculator()

In [7]:
params = metric_calculator.get_default_params_dict()

In [8]:
start_epoch = Epoch("start", 0, 15)
end_epoch = Epoch("end", 15, 30)
epochs = [start_epoch, end_epoch]
unit_ids = sorting.get_unit_ids()#[1,2,3,12,22]

In [9]:
metrics_pd = metrics.calculate_metrics(spike_times=spike_times, spike_clusters=spike_clusters, amplitudes=amplitudes, 
                                       pc_features=pc_features, pc_feature_ind=pc_feature_ind, params=params, epochs = epochs, cluster_ids=sorting.get_unit_ids())

Calculating isi violations
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
Calculating presence ratio
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
Calculating firing rate
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
Calculating amplitude cutoff
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
Calculating PC-based metrics
 ▒▒▒▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ 19% 

/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1807: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= _b) & cond0


 ▒▒▒▒▒▒▒▒▒▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░ 33% 

/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1807: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= _b) & cond0


 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░ 41% 

...

/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1807: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= _b) & cond0


 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒░░░░░ 89% 

/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1807: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= _b) & cond0


 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
Calculating silhouette score
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
Calculating drift metrics
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 


/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/spikemetrics-0.1.0-py3.7.egg/spikemetrics/metrics.py:374: RuntimeWarning: All-NaN slice encountered


In [10]:
metric_calculator.store_all_metric_data(recording, sorting, nPC=3, ms_before=1., ms_after=2., dtype=None, max_num_waveforms=np.inf, \
                                        max_num_pca_waveforms=np.inf, save_features_props=False)


In [11]:
epoch_tuples = [(start_epoch.start_time, start_epoch.end_time), (end_epoch.start_time, end_epoch.end_time)]
epoch_names = ["start", "end"]

In [12]:
firing_rates = metric_calculator.compute_firing_rates(unit_ids=unit_ids, epoch_tuples=epoch_tuples, epoch_names=epoch_names)

num_spikes = metric_calculator.compute_num_spikes(unit_ids=unit_ids, epoch_tuples=epoch_tuples)

isi_violations = metric_calculator.compute_isi_violations(isi_threshold=0.0015, min_isi=0.000166, unit_ids=unit_ids, epoch_tuples=epoch_tuples, epoch_names=epoch_names)

presence_ratios = metric_calculator.compute_presence_ratios(unit_ids=unit_ids, epoch_tuples=epoch_tuples, epoch_names=epoch_names)

max_drifts, cumulative_drifts = metric_calculator.compute_drift_metrics(drift_metrics_interval_s=51, drift_metrics_min_spikes_per_interval=10, unit_ids=unit_ids, \
                                                                        epoch_tuples=epoch_tuples)
silhouette_scores = metric_calculator.compute_silhouette_score(unit_ids=unit_ids, epoch_tuples=epoch_tuples, epoch_names=epoch_names, seed=0)

isolation_distances = metric_calculator.compute_isolation_distances(num_channels_to_compare=13, max_spikes_for_unit=500, unit_ids=unit_ids, epoch_tuples=epoch_tuples, epoch_names=epoch_names, seed=0)

l_ratios = metric_calculator.compute_l_ratios(num_channels_to_compare=13, max_spikes_for_unit=500, unit_ids=unit_ids, epoch_tuples=epoch_tuples, epoch_names=epoch_names, seed=0)

d_primes = metric_calculator.compute_d_primes(num_channels_to_compare=13, max_spikes_for_unit=500, unit_ids=unit_ids, epoch_tuples=epoch_tuples, epoch_names=epoch_names, seed=0)

nn_hit_rates, nn_miss_rates = metric_calculator.compute_nn_metrics(num_channels_to_compare=13, max_spikes_for_unit=500, max_spikes_for_nn=10000, n_neighbors=4, unit_ids=unit_ids, epoch_tuples=epoch_tuples, epoch_names=epoch_names, seed=0)

 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ 11% 

/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/spikemetrics-0.1.0-py3.7.egg/spikemetrics/metrics.py:374: RuntimeWarning: All-NaN slice encountered


 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒░░░░░░░░░░░░░░░ 63% 

/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/spikemetrics-0.1.0-py3.7.egg/spikemetrics/metrics.py:374: RuntimeWarning: All-NaN slice encountered
/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1807: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= _b) & cond0


 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 


In [13]:
def nan_equal(a,b):
    try:
        np.testing.assert_equal(a,b)
    except AssertionError:
        return False
    return True

In [14]:
metrics_dict = metric_calculator.get_metrics_dict()

print(nan_equal(np.asarray([np.hstack(epoch_metrics[2]) for epoch_metrics in metrics_dict['firing_rate']]).flatten(), np.asarray(metrics_pd.loc[metrics_pd['cluster_id'].isin(unit_ids)]['firing_rate'])))
print(nan_equal(np.asarray([np.hstack(epoch_metrics[2]) for epoch_metrics in metrics_dict['num_spikes']]).flatten(), np.asarray(metrics_pd.loc[metrics_pd['cluster_id'].isin(unit_ids)]['num_spikes'])))
print(nan_equal(np.asarray([np.hstack(epoch_metrics[2]) for epoch_metrics in metrics_dict['isi_viol']]).flatten(), np.asarray(metrics_pd.loc[metrics_pd['cluster_id'].isin(unit_ids)]['isi_viol'])))
print(nan_equal(np.asarray([np.hstack(epoch_metrics[2]) for epoch_metrics in metrics_dict['presence_ratio']]).flatten(), np.asarray(metrics_pd.loc[metrics_pd['cluster_id'].isin(unit_ids)]['presence_ratio'])))
print(nan_equal(np.asarray([np.hstack(epoch_metrics[2]) for epoch_metrics in metrics_dict['max_drift']]).flatten(), np.asarray(metrics_pd.loc[metrics_pd['cluster_id'].isin(unit_ids)]['max_drift'])))
print(nan_equal(np.asarray([np.hstack(epoch_metrics[2]) for epoch_metrics in metrics_dict['cumulative_drift']]).flatten(), np.asarray(metrics_pd.loc[metrics_pd['cluster_id'].isin(unit_ids)]['cumulative_drift'])))
print(nan_equal(np.asarray([np.hstack(epoch_metrics[2]) for epoch_metrics in metrics_dict['silhouette_score']]).flatten(), np.asarray(metrics_pd.loc[metrics_pd['cluster_id'].isin(unit_ids)]['silhouette_score'])))
print(nan_equal(np.asarray([np.hstack(epoch_metrics[2]) for epoch_metrics in metrics_dict['isolation_distance']]).flatten(), np.asarray(metrics_pd.loc[metrics_pd['cluster_id'].isin(unit_ids)]['isolation_distance'])))
print(nan_equal(np.asarray([np.hstack(epoch_metrics[2]) for epoch_metrics in metrics_dict['l_ratio']]).flatten(), np.asarray(metrics_pd.loc[metrics_pd['cluster_id'].isin(unit_ids)]['l_ratio'])))
print(nan_equal(np.asarray([np.hstack(epoch_metrics[2]) for epoch_metrics in metrics_dict['d_prime']]).flatten(), np.asarray(metrics_pd.loc[metrics_pd['cluster_id'].isin(unit_ids)]['d_prime'])))
print(nan_equal(np.asarray([np.hstack(epoch_metrics[2]) for epoch_metrics in metrics_dict['nn_hit_rate']]).flatten(), np.asarray(metrics_pd.loc[metrics_pd['cluster_id'].isin(unit_ids)]['nn_hit_rate'])))
print(nan_equal(np.asarray([np.hstack(epoch_metrics[2]) for epoch_metrics in metrics_dict['nn_miss_rate']]).flatten(), np.asarray(metrics_pd.loc[metrics_pd['cluster_id'].isin(unit_ids)]['nn_miss_rate'])))

True
True
True
True
True
True
True
True
True
True
True
True


In [15]:
metric_names = ['firing_rate', 'num_spikes', 'isi_viol', 'presence_ratio', 'max_drift', 'cumulative_drift', 'silhouette_score', 'isolation_distance', \
                'l_ratio', 'd_prime', 'nn_hit_rate', 'nn_miss_rate']

metrics_epochs = metric_calculator.compute_metrics(isi_threshold=0.0015, min_isi=0.000166, drift_metrics_interval_s=51, \
                                                   drift_metrics_min_spikes_per_interval=10, max_spikes_for_silhouette=10000, \
                                                   num_channels_to_compare=13, max_spikes_for_unit=500, max_spikes_for_nn=10000, \
                                                   n_neighbors=4, metric_names=metric_names, unit_ids=unit_ids, epoch_tuples=epoch_tuples, 
                                                   epoch_names=epoch_names, seed=0)

 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ 11% 

/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/spikemetrics-0.1.0-py3.7.egg/spikemetrics/metrics.py:374: RuntimeWarning: All-NaN slice encountered


 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒░░░░░░░░░░░░ 70% 

/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/spikemetrics-0.1.0-py3.7.egg/spikemetrics/metrics.py:374: RuntimeWarning: All-NaN slice encountered
/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1807: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= _b) & cond0


 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 


In [16]:
print(nan_equal(np.hstack(metrics_epochs[0]).flatten(), np.asarray(metrics_pd.loc[metrics_pd['cluster_id'].isin(unit_ids)]['firing_rate'])))
print(nan_equal(np.hstack(metrics_epochs[1]).flatten(), np.asarray(metrics_pd.loc[metrics_pd['cluster_id'].isin(unit_ids)]['num_spikes'])))
print(nan_equal(np.hstack(metrics_epochs[2]).flatten(), np.asarray(metrics_pd.loc[metrics_pd['cluster_id'].isin(unit_ids)]['isi_viol'])))
print(nan_equal(np.hstack(metrics_epochs[3]).flatten(), np.asarray(metrics_pd.loc[metrics_pd['cluster_id'].isin(unit_ids)]['presence_ratio'])))
print(nan_equal(np.hstack(metrics_epochs[4]).flatten(), np.asarray(metrics_pd.loc[metrics_pd['cluster_id'].isin(unit_ids)]['max_drift'])))
print(nan_equal(np.hstack(metrics_epochs[5]).flatten(), np.asarray(metrics_pd.loc[metrics_pd['cluster_id'].isin(unit_ids)]['cumulative_drift'])))
print(nan_equal(np.hstack(metrics_epochs[6]).flatten(), np.asarray(metrics_pd.loc[metrics_pd['cluster_id'].isin(unit_ids)]['silhouette_score'])))
print(nan_equal(np.hstack(metrics_epochs[7]).flatten(), np.asarray(metrics_pd.loc[metrics_pd['cluster_id'].isin(unit_ids)]['isolation_distance'])))
print(nan_equal(np.hstack(metrics_epochs[8]).flatten(), np.asarray(metrics_pd.loc[metrics_pd['cluster_id'].isin(unit_ids)]['l_ratio'])))
print(nan_equal(np.hstack(metrics_epochs[9]).flatten(), np.asarray(metrics_pd.loc[metrics_pd['cluster_id'].isin(unit_ids)]['d_prime'])))
print(nan_equal(np.hstack(metrics_epochs[10]).flatten(), np.asarray(metrics_pd.loc[metrics_pd['cluster_id'].isin(unit_ids)]['nn_hit_rate'])))
print(nan_equal(np.hstack(metrics_epochs[11]).flatten(), np.asarray(metrics_pd.loc[metrics_pd['cluster_id'].isin(unit_ids)]['nn_miss_rate'])))

True
True
True
True
True
True
True
True
True
True
True
True


In [40]:
import pandas as pd
metrics_df = pd.DataFrame()

In [38]:
for metric_epoch in metrics_dict['firing_rate']:
    print(metric_epoch[0])
    print(metric_epoch[1])
    print(metric_epoch[2])
    print("")

('start', 0, 15)
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
[ 2.26827336  5.20368594 15.34420214  1.53442021  2.20155944  1.46770629
 11.20793896 13.94320977  3.20226827  4.26969103  0.66713922  9.00637952
 16.21148313  2.00141767 19.88074886  2.53512905  4.4698328  11.20793896
  3.33569612  6.60467831  7.20510362  5.40382771  1.40099237 11.67493641
 15.2107743  13.47621232 11.74165034]

('end', 15, 30)
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
[ 2.60109463  3.80159984 13.40564154  1.66736835  4.20176824  0.86703154
 10.33768378 15.40648356  5.33557872  4.06837878  0.86703154  8.47023122
 15.6732625   2.26762096 16.00673617  3.06795777  4.73532612  8.27014702
  2.20092622  6.4693892   9.00378909  5.66905239  1.26719995 11.73827319
 13.73911521 12.53861    11.27141005]



In [54]:
from collections import OrderedDict

In [62]:
pd.DataFrame(data=OrderedDict((('unit_ids' , metrics_dict['firing_rate'][0][1]),
                                ('firing_rate' , metrics_dict['firing_rate'][0][2]),
                                )))

,unit_ids,firing_rate
0,0,2.268273
1,1,5.203686
2,2,15.344202
3,3,1.534420
4,4,2.201559
5,5,1.467706
6,6,11.207939
7,7,13.943210
8,8,3.202268
9,9,4.269691


In [57]:
metrics_dict['firing_rate'][1]

(('end', 15, 30),
 [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26],
 array([ 2.60109463,  3.80159984, 13.40564154,  1.66736835,  4.20176824,
         0.86703154, 10.33768378, 15.40648356,  5.33557872,  4.06837878,
         0.86703154,  8.47023122, 15.6732625 ,  2.26762096, 16.00673617,
         3.06795777,  4.73532612,  8.27014702,  2.20092622,  6.4693892 ,
         9.00378909,  5.66905239,  1.26719995, 11.73827319, 13.73911521,
        12.53861   , 11.27141005]))